In [1]:
import numpy as np

real_data = np.load('real.npz')
fake_data = np.load('fake.npz')

C:\Users\dell\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\dell\anaconda3\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
C:\Users\dell\anaconda3\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
real_data.keys(), fake_data.keys()

(KeysView(<numpy.lib.npyio.NpzFile object at 0x000001F172B27190>),
 KeysView(<numpy.lib.npyio.NpzFile object at 0x000001F172B27100>))

In [3]:
real_x = real_data['x']
real_y = real_data['y']

fake_x = fake_data['x']
fake_y = fake_data['y']

In [4]:
real_x.shape, type(real_x)

((750, 1998, 101), numpy.ndarray)

In [5]:
fake_x.shape, type(fake_x)

((750, 1998, 101), numpy.ndarray)

In [6]:
real_y.shape, type(real_y)

((750, 496), numpy.ndarray)

In [7]:
fake_y.shape, type(fake_y)

((750, 496), numpy.ndarray)

In [8]:
np.unique(real_y), np.unique(fake_y)

(array([0, 1], dtype=int8), array([0], dtype=int8))

In [9]:
np.all(np.logical_and(np.any(real_y == 0, axis=1), np.any(real_y == 1, axis=1)))

True

In [10]:
np.any(np.isnan(real_x.flatten())), np.any(np.isnan(fake_x.flatten()))

(False, False)

In [11]:
np.any(np.isinf(real_x.flatten())), np.any(np.isinf(fake_x.flatten()))

(True, True)

In [12]:
import numpy as np
from sklearn.model_selection import train_test_split

X = np.concatenate((real_x, fake_x), axis=0)
Y = np.concatenate((real_y, fake_y), axis=0)

# X[np.isinf(X)] = np.finfo(X.dtype).max
Y = np.any(Y == 1, axis=-1)

rng = np.random.default_rng(seed=42)
indices = np.arange(X.shape[0])
rng.shuffle(indices)

X_shuffled = X[indices]
Y_shuffled = Y[indices]

X_train, X_temp, Y_train, Y_temp = train_test_split(X_shuffled, Y_shuffled, test_size=0.2, random_state=42)
X_test, X_val, Y_test, Y_val = train_test_split(X_temp, Y_temp, test_size=0.5, random_state=42)

In [13]:
X_train.shape, Y_train.shape

((1200, 1998, 101), (1200,))

In [14]:
np.finfo(X.dtype).max

65500.0

In [15]:
def handle_infinite_values(x):
    return tf.where(x > 65500, tf.fill(tf.shape(x), tf.constant(65500, dtype=x.dtype)), x)

In [16]:
import tensorflow as tf

model = tf.keras.Sequential([
    tf.keras.layers.Lambda(handle_infinite_values, input_shape=(1998, 101)),
    tf.keras.layers.Conv1D(64, kernel_size=3, activation='relu'),
    tf.keras.layers.MaxPooling1D(pool_size=2),
    tf.keras.layers.Conv1D(128, kernel_size=3, activation='relu'),
    tf.keras.layers.MaxPooling1D(pool_size=2),
    tf.keras.layers.Conv1D(256, kernel_size=3, activation='relu'),
    tf.keras.layers.MaxPooling1D(pool_size=2),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Conv1D(512, kernel_size=3, activation='relu'),
    tf.keras.layers.MaxPooling1D(pool_size=2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [17]:
acc_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath='best_accuracy_weights.h5',
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True,
)

loss_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath='best_loss_weights.h5',
    save_weights_only=True,
    monitor='val_loss',
    mode='min',
    save_best_only=True,
)

hist = model.fit(X_train, Y_train,
                 epochs=30, batch_size=32,
                 validation_data=(X_val, Y_val),
                 callbacks=[acc_checkpoint, loss_checkpoint]
                )

Epoch 1/30
38/38 [==============================] - 37s 883ms/step - loss: 156.8828 - accuracy: 0.5133 - val_loss: 0.6903 - val_accuracy: 0.5267
Epoch 2/30
38/38 [==============================] - 33s 868ms/step - loss: 0.8231 - accuracy: 0.5533 - val_loss: 0.6917 - val_accuracy: 0.5067
Epoch 3/30
38/38 [==============================] - 31s 816ms/step - loss: 0.7136 - accuracy: 0.5767 - val_loss: 0.6745 - val_accuracy: 0.5267
Epoch 4/30
38/38 [==============================] - 33s 869ms/step - loss: 0.5741 - accuracy: 0.7017 - val_loss: 0.4950 - val_accuracy: 0.7600
Epoch 5/30
38/38 [==============================] - 31s 820ms/step - loss: 0.4359 - accuracy: 0.7917 - val_loss: 0.3286 - val_accuracy: 0.8267
Epoch 6/30
38/38 [==============================] - 31s 822ms/step - loss: 0.3432 - accuracy: 0.8483 - val_loss: 0.2299 - val_accuracy: 0.8933
Epoch 7/30
38/38 [==============================] - 34s 897ms/step - loss: 0.2526 - accuracy: 0.8975 - val_loss: 0.1690 - val_accuracy: 0.96

In [18]:
# model.load_weights('best_accuracy_weights.h5')
model.load_weights('best_loss_weights.h5')

In [19]:
model.evaluate(X_val, Y_val)

5/5 [==============================] - 1s 99ms/step - loss: 0.0036 - accuracy: 1.0000


[0.0035534945782274008, 1.0]

In [20]:
model.evaluate(X_test, Y_test)

5/5 [==============================] - 1s 102ms/step - loss: 0.0335 - accuracy: 0.9867


[0.03351197391748428, 0.9866666793823242]